<a href="https://colab.research.google.com/github/Vignesh1725/women_safety_analysis_codes/blob/main/DeepSeekBestTrainCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install ultralytics opencv-python numpy scikit-learn albumentations matplotlib torchmetrics
!pip install -U kaggle

In [6]:
# Dataset setup
import os
if not os.path.exists('/content/utkface'):
    from google.colab import files
    files.upload()  # Upload kaggle.json
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    !kaggle datasets download -d jangedoo/utkface-new
    !unzip utkface-new.zip -d utkface


In [7]:
import cv2
import numpy as np
import albumentations as A
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torch
from torchmetrics import Accuracy
import matplotlib.pyplot as plt

In [10]:
# At the start of your script
import torch
if torch.cuda.is_available():
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    print("Using CPU")

Using GPU: Tesla T4


In [11]:
# At the start of your script
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [12]:


class Config:
    SEED = 42
    IMG_SIZE = 224
    BATCH_SIZE = 64
    EPOCHS = 50
    DATA_PATH = '/content/utkface/UTKFace'
    OUTPUT_PATH = '/content/gender_dataset'
    MODEL_NAME = 'yolov8m-cls'
    DEVICE = 0 if torch.cuda.is_available() else 'cpu'  # Changed from string '0' to integer 0
    TORCH_DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')  # Added for PyTorch operations

# Set seed for reproducibility
torch.manual_seed(Config.SEED)
np.random.seed(Config.SEED)

def prepare_dataset():
    """Enhanced dataset preparation with distance-aware augmentations"""
    os.makedirs(f'{Config.OUTPUT_PATH}/train/male', exist_ok=True)
    os.makedirs(f'{Config.OUTPUT_PATH}/train/female', exist_ok=True)
    os.makedirs(f'{Config.OUTPUT_PATH}/val/male', exist_ok=True)
    os.makedirs(f'{Config.OUTPUT_PATH}/val/female', exist_ok=True)

    images = [f for f in os.listdir(Config.DATA_PATH) if f.endswith('.jpg')]
    genders = [0 if img.split('_')[1] == '0' else 1 for img in images]

    train, val = train_test_split(images, test_size=0.2, stratify=genders, random_state=Config.SEED)

    # Core augmentation pipeline
    def get_augmentations(gender, is_train):
        transforms = [
            A.Resize(Config.IMG_SIZE, Config.IMG_SIZE),
            A.HorizontalFlip(p=0.5),
            A.RandomBrightnessContrast(p=0.3)
        ]

        if is_train:
            transforms.extend([
                A.GaussianBlur(blur_limit=(3, 7), p=0.2),
                A.RandomScale(scale_limit=(-0.5, -0.2)),  # Distance simulation
                A.PadIfNeeded(min_height=Config.IMG_SIZE, min_width=Config.IMG_SIZE),
                A.RandomCrop(height=Config.IMG_SIZE, width=Config.IMG_SIZE),
                A.CoarseDropout(max_holes=8, p=0.3)
            ])

            if gender == 'male':
                transforms.append(A.RandomShadow(p=0.2))
            else:
                transforms.append(A.CLAHE(p=0.2))

        return A.Compose(transforms)

    for split, files in [('train', train), ('val', val)]:
        gender_counts = {'male': 0, 'female': 0}
        for img in files:
            try:
                gender = 'male' if img.split('_')[1] == '0' else 'female'
                img_path = os.path.join(Config.DATA_PATH, img)
                img_arr = cv2.cvtColor(cv2.imread(img_path), cv2.COLOR_BGR2RGB)

                augmenter = get_augmentations(gender, split == 'train')
                augmented = augmenter(image=img_arr)['image']

                cv2.imwrite(
                    os.path.join(Config.OUTPUT_PATH, split, gender, img),
                    cv2.cvtColor(augmented, cv2.COLOR_RGB2BGR)
                )
                gender_counts[gender] += 1
            except Exception as e:
                print(f"Skipping {img}: {str(e)}")

        print(f"{split} set distribution: {gender_counts}")

prepare_dataset()

def train_model():
    """Enhanced training with metrics tracking"""
    model = YOLO(f'{Config.MODEL_NAME}.pt')

    class TrainingMonitor:
        def __init__(self):
            self.accuracy = Accuracy(task='binary').to(Config.TORCH_DEVICE)
            self.best_acc = 0
            self.history = {'loss': [], 'val_loss': [], 'acc': [], 'val_acc': []}

        def on_epoch_end(self, trainer):
            current_acc = self.accuracy.compute().item()
            self.history['loss'].append(trainer.training_loss)
            self.history['val_loss'].append(trainer.validation_loss)
            self.history['acc'].append(current_acc)

            if current_acc > self.best_acc:
                self.best_acc = current_acc
                trainer.save('best.pt')

            self.accuracy.reset()
            self.plot_progress()

        def plot_progress(self):
            plt.figure(figsize=(12, 5))
            plt.subplot(1, 2, 1)
            plt.plot(self.history['loss'], label='Train')
            plt.plot(self.history['val_loss'], label='Validation')
            plt.title('Loss')
            plt.legend()

            plt.subplot(1, 2, 2)
            plt.plot(self.history['acc'], label='Train')
            plt.title('Accuracy')
            plt.legend()
            plt.show()

    monitor = TrainingMonitor()

    try:
        model.train(
            data=Config.OUTPUT_PATH,
            epochs=Config.EPOCHS,
            imgsz=Config.IMG_SIZE,
            batch=Config.BATCH_SIZE,
            device=Config.DEVICE,
            optimizer='AdamW',
            lr0=0.001,
            cos_lr=True,
            augment=True,
            dropout=0.2,
            label_smoothing=0.1,
            warmup_epochs=3,
            callbacks={'on_epoch_end': monitor.on_epoch_end}
        )
    except Exception as e:
        print(f"Training error: {e}")
        model.train(
            data=Config.OUTPUT_PATH,
            epochs=30,
            imgsz=160,
            batch=32,
            device=Config.DEVICE,
            lr0=0.0005,
            warmup_epochs=3
        )

    model.export(format='onnx')
    print("Training complete! Model exported to ONNX format.")

train_model()

<ipython-input-12-553eb3e2c5e2>:42: UserWarning: Argument(s) 'max_holes' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, p=0.3)


train set distribution: {'male': 9913, 'female': 9053}
val set distribution: {'male': 2478, 'female': 2264}
New https://pypi.org/project/ultralytics/8.3.139 available 😃 Update with 'pip install -U ultralytics'
WARNING ⚠️ 'label_smoothing' is deprecated and will be removed in in the future.
Training error: 'callbacks' is not a valid YOLO argument. 

    Arguments received: ['yolo', '-f', '/root/.local/share/jupyter/runtime/kernel-dc7b5768-1cdd-4005-bacd-f80254902d7b.json']. Ultralytics 'yolo' commands use the following syntax:

        yolo TASK MODE ARGS

        Where   TASK (optional) is one of frozenset({'obb', 'pose', 'segment', 'detect', 'classify'})
                MODE (required) is one of frozenset({'predict', 'train', 'export', 'benchmark', 'val', 'track'})
                ARGS (optional) are any number of custom 'arg=value' pairs like 'imgsz=320' that override defaults.
                    See all ARGS at https://docs.ultralytics.com/usage/cfg or with 'yolo cfg'

    1. Train

100%|██████████| 5.35M/5.35M [00:00<00:00, 127MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 538.5±306.5 MB/s, size: 11.0 KB)


train: Scanning /content/gender_dataset/train... 18966 images, 0 corrupt: 100%|██████████| 18966/18966 [00:03<00:00, 5714.72it/s]


...

      30/30      3.54G     0.0935         22        160: 100%|██████████| 593/593 [01:13<00:00,  8.11it/s]
               classes   top1_acc   top5_acc: 100%|██████████| 75/75 [00:07<00:00, 10.09it/s]

                   all      0.942          1



30 epochs completed in 0.677 hours.
Optimizer stripped from runs/classify/train/weights/last.pt, 31.7MB
Optimizer stripped from runs/classify/train/weights/best.pt, 31.7MB

Validating runs/classify/train/weights/best.pt...
Ultralytics 8.3.138 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLOv8m-cls summary (fused): 42 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs
train: /content/gender_dataset/train... found 18966 images in 2 classes ✅ 
val: /content/gender_dataset/val... found 4742 images in 2 classes ✅ 
test: None...


               classes   top1_acc   top5_acc: 100%|██████████| 75/75 [00:08<00:00,  8.92it/s]


                   all      0.946          1
Speed: 0.0ms preprocess, 0.5ms inference, 0.0ms loss, 0.0ms postprocess per image
Results saved to runs/classify/train
Ultralytics 8.3.138 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.00GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
YOLOv8m-cls summary (fused): 42 layers, 15,765,218 parameters, 0 gradients, 41.6 GFLOPs

PyTorch: starting from 'runs/classify/train/weights/best.pt' with input shape (1, 3, 160, 160) BCHW and output shape(s) (1, 2) (30.2 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.53', 'onnxruntime-gpu'] not found, attempting AutoUpdate...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 305.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.2/146.2 kB 355.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.2/283.2 MB 290.5 MB/s eta 0:00:00
   ━━━━━━━━

In [15]:
from google.colab import files

# Download the best model
files.download('/content/runs/classify/train/weights/best.pt')  # Default YOLOv8 save path
files.download('/content/runs/classify/train/weights/last.pt')
files.download('/content/runs/classify/train/weights/best.onnx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>